### 사전 준비

In [1]:
# transformers 및 데이터셋(NSMC) 설치

!pip install transformers
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.0 MB/s eta 0:00:00
--2023-02-15 06:37:16--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  --.-KB/s    in 0.03s   

2023-02-15 06:37:16 (162 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2023-02-15 06:37:16--  https://raw.githubusercontent.com/e9t/nsmc/maste

In [2]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
from transformers import ElectraModel, ElectraTokenizer
import random
import os
import numpy as np

In [3]:
torch.cuda.empty_cache()

device = torch.device("cuda:0")
print(device)

cuda:0


In [4]:
def fix_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

my_seed = 42
fix_seed(my_seed)

In [5]:
def seed_worker(_worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

### 데이터 load 및 증강

In [6]:
filenames = ['ratings_train.txt', 'ratings_test.txt']
with open('rating.txt', 'w') as outfile:
    for filename in filenames:
        with open(filename) as file:
            outfile.write(file.read())

In [7]:
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file,idx1, idx2):
    question = ['부정적인', '긍정적인'] # pseudo question
    self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
    self.dataset.drop_duplicates(subset=['document'], inplace=True)
    self.dataset=self.dataset.sample(frac=1, random_state=my_seed).reset_index(drop=True)
    self.dataset = self.dataset[idx1:idx2]

    self.document = self.dataset['document'].tolist()
    self.label = self.dataset['label'].astype(int).tolist()

    # document를 질문을 포함하도록 수정하고, label을 그 질문에 대한 label로 변환하는 질문 형태의 증강 기법을 사용해 데이터 증강
    self.data = []
    for i,j in zip(self.document, self.label):
      for e, q in enumerate(question):
        self.data.append([i+'[SEP]'+q, 1 if j==e else 0])
    
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, idx):
    text = self.data[idx][0]
    y = self.data[idx][1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=128,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [8]:
train_dataset = NSMCDataset("rating.txt",0,8000)
val_dataset = NSMCDataset("rating.txt",8000,9000)
test_dataset = NSMCDataset("rating.txt",9000,10000)

In [9]:
# 데이터 예시

train_dataset[1]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(tensor([    2,   889, 29951, 26639,  8927, 29956, 30433, 30040,  5036, 29950,
         30090,   131, 27134, 29992,   191,  4632, 29951, 29998, 30407, 30726,
         29986,   889,  8927, 31271, 30582, 29966, 30254, 30326,     3,  2411,
         30006, 29972,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

### 실제 데이터셋 증강 예시<br>
data_ex = ["아 더빙.. 진짜 짜증나네요 목소리",0] <br>
-> [["아 더빙.. 진짜 짜증나네요 목소리 [SEP] 부정적인",1],["아 더빙.. 진짜 짜증나네요 목소리 [SEP] 긍정적인",0]]

### 모델 load 및 fine-tuning

In [10]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [11]:
epochs = 5
batch_size =64

In [12]:
g = torch.Generator()
g.manual_seed(0)

optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, worker_init_fn=seed_worker, generator=g)
val_loader = DataLoader(val_dataset, batch_size=batch_size, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_dataset, batch_size=batch_size, worker_init_fn=seed_worker, generator=g)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
losses = []
accuracies = []

for i in range(epochs):
  print("train {} epochs start!".format(i+1))
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0
  val_batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 40 == 0:
      print("Batch Loss:", loss, "Accuracy:", correct.float() / total)
    losses.append(loss)

  print("validation start!")
  for input_ids_batch, attention_masks_batch, y_batch in tqdm(val_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    val_batches += 1
    if val_batches % 10 == 0:
      print("Batch Loss:", loss, "Accuracy:", correct.float() / total)

  accuracies.append(correct.float() / total)
  print("Train Loss:", loss, "Accuracy:", correct.float() / total)

train 1 epochs start!


  0%|          | 0/250 [00:00<?, ?it/s]

Batch Loss: tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5043, device='cuda:0')
Batch Loss: tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5066, device='cuda:0')
Batch Loss: tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5057, device='cuda:0')
Batch Loss: tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5079, device='cuda:0')
Batch Loss: tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5116, device='cuda:0')
Batch Loss: tensor(0.6866, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5316, device='cuda:0')
validation start!


  0%|          | 0/32 [00:00<?, ?it/s]

Batch Loss: tensor(0.6738, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5441, device='cuda:0')
Batch Loss: tensor(0.6727, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5501, device='cuda:0')
Batch Loss: tensor(0.6720, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5557, device='cuda:0')
Train Loss: tensor(0.6567, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.5563, device='cuda:0')
train 2 epochs start!


...

Batch Loss: tensor(0.5181, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8305, device='cuda:0')
Batch Loss: tensor(0.3587, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8313, device='cuda:0')
Batch Loss: tensor(0.5143, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8305, device='cuda:0')
Train Loss: tensor(0.5353, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8301, device='cuda:0')
train 5 epochs start!


  0%|          | 0/250 [00:00<?, ?it/s]

Batch Loss: tensor(0.4966, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8465, device='cuda:0')
Batch Loss: tensor(0.3521, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8422, device='cuda:0')
Batch Loss: tensor(0.4096, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8477, device='cuda:0')
Batch Loss: tensor(0.5191, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8442, device='cuda:0')
Batch Loss: tensor(0.3947, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8448, device='cuda:0')
Batch Loss: tensor(0.6209, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8457, device='cuda:0')
validation start!


  0%|          | 0/32 [00:00<?, ?it/s]

Batch Loss: tensor(0.5558, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8443, device='cuda:0')
Batch Loss: tensor(0.3501, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8438, device='cuda:0')
Batch Loss: tensor(0.5020, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8424, device='cuda:0')
Train Loss: tensor(0.3805, device='cuda:0', grad_fn=<NllLossBackward0>) Accuracy: tensor(0.8421, device='cuda:0')


### Test data를 사용한 평가

In [14]:
model.eval()

predict = []
test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  predicts = []
  for i in predicted:
    predicts.append(i.item())
  predict.extend(predicts)

  0%|          | 0/32 [00:00<?, ?it/s]

In [15]:
# 예측 라벨을 기존 형태로 변환

import numpy as np
result = []
s=0
for i in range(2,len(predict)+1,2):
    result.append(np.argmax(predict[s:i]))
    s=i

In [16]:
# 예측 성능 평가

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, confusion_matrix

print("accuracy:{}".format(accuracy_score(test_dataset.label, result)))
print("f1-score:{}".format(f1_score(test_dataset.label, result)))
print("precision:{}".format(precision_score(test_dataset.label, result ,pos_label=1)))
print("recall:{}".format(recall_score(test_dataset.label, result ,pos_label=1)))

accuracy:0.838
f1-score:0.8329896907216495
precision:0.8523206751054853
recall:0.8145161290322581
